<a href="https://colab.research.google.com/github/ZiadHelal/llm-finetuning-demo/blob/main/LanguageModel_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Packages

In [1]:
!pip install --upgrade lamini
!pip install --upgrade datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


# Importing Libraries

In [2]:
from rich import print
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

# Loading Lamini docs dataset

In [7]:
dataset_path = "lamini/lamini_docs"
use_hf = True

# Set up the model, training config, and tokenizer

In [8]:
model_name = "EleutherAI/pythia-70m"

In [9]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

DEBUG:utilities:Config: datasets.path: lamini/lamini_docs
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize True lamini/lamini_docs


DEBUG:fsspec:<File-like object HfFileSystem, datasets/lamini/lamini_docs@05bd680b81d69a7a1d38193873f1487d73e535bf/data/train-00000-of-00001-5cdebbc48da41394.parquet> read: 0 - 4194304
DEBUG:fsspec:<File-like object HfFileSystem, datasets/lamini/lamini_docs@05bd680b81d69a7a1d38193873f1487d73e535bf/data/train-00000-of-00001-5cdebbc48da41394.parquet> read: 614936 - 4809240


DEBUG:fsspec:<File-like object HfFileSystem, datasets/lamini/lamini_docs@05bd680b81d69a7a1d38193873f1487d73e535bf/data/test-00000-of-00001-4c77a066a883f339.parquet> read: 0 - 4194304
DEBUG:fsspec:<File-like object HfFileSystem, datasets/lamini/lamini_docs@05bd680b81d69a7a1d38193873f1487d73e535bf/data/test-00000-of-00001-4c77a066a883f339.parquet> read: 83671 - 4277975


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

...

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default-9b991800e664930e/0.0.0/ca31c69184d9832faed373922c2acccec0b13a0bb5bbbe19371385c3ff26f1d1.incomplete/lamini_docs-train-00000-00000-of-NNNNN.arrow


Generating test split:   0%|          | 0/140 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default-9b991800e664930e/0.0.0/ca31c69184d9832faed373922c2acccec0b13a0bb5bbbe19371385c3ff26f1d1.incomplete/lamini_docs-test-00000-00000-of-NNNNN.arrow
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default-9b991800e664930e/0.0.0/ca31c69184d9832faed373922c2acccec0b13a0bb5bbbe19371385c3ff26f1d1.incomplete/dataset_info.json


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})